In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(f"comentarios_todos_modelar.csv")

In [ ]:
from tqdm import tqdm
import time

In [ ]:
df

,index,category,comment
0,6654,ideología política,Feaaaaaaaaaaaaa comunista
1,6019,racismo,Es una gallega oligarca 🤦
2,124,machismo,Esta tipa es una vulgar
3,6231,ideología política,Cállate petarda que eres una corderita una co...
4,5409,ideología política,"TODA RIDICULA GERRA ,,,, SOLAMENTE TRAE MAS PE..."
...,...,...,...
1003,568,racismo,Esta mujer venezolana debio pensarlo y meditar...
1004,2143,ideología política,En España preferimos inmigrantes hispaoamerica...
1005,712,racismo,La mayoria de los centroamericanos y sudameric...
1006,1436,ideología política,Todos los inmigrantes que paguen lo que paga u...


In [ ]:
df['category'].value_counts()

ideología política    357
racismo               337
machismo              314
Name: category, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['num_cat']= label_encoder.fit_transform(df['category'])


In [ ]:
df.num_cat.unique() # ideologia politica = 0 , racismo= 2 y machismo = 1

array([0, 2, 1])

In [ ]:
# Aplicar one-hot encoding
one_hot = pd.get_dummies(df['category'])

# Concatenar las columnas codificadas one-hot al DataFrame original
df = pd.concat([df, one_hot], axis=1)


In [ ]:
df

,index,category,comment,num_cat,ideología política,machismo,racismo
0,6654,ideología política,Feaaaaaaaaaaaaa comunista,0,1,0,0
1,6019,racismo,Es una gallega oligarca 🤦,2,0,0,1
2,124,machismo,Esta tipa es una vulgar,1,0,1,0
3,6231,ideología política,Cállate petarda que eres una corderita una co...,0,1,0,0
4,5409,ideología política,"TODA RIDICULA GERRA ,,,, SOLAMENTE TRAE MAS PE...",0,1,0,0
...,...,...,...,...,...,...,...
1003,568,racismo,Esta mujer venezolana debio pensarlo y meditar...,2,0,0,1
1004,2143,ideología política,En España preferimos inmigrantes hispaoamerica...,0,1,0,0
1005,712,racismo,La mayoria de los centroamericanos y sudameric...,2,0,0,1
1006,1436,ideología política,Todos los inmigrantes que paguen lo que paga u...,0,1,0,0


In [ ]:
#df.to_csv('comentarios_todos_modelar_num_cat.csv', index=False)  # index=False evita que se escriba el índice del DataFrame en el archivo


import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Convertir la Serie de Pandas a una lista de strings
comentarios = df['comment'].tolist()
categorias = df['num_cat'].tolist()

# Codificación de Etiquetas
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(categorias)

# Tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Codificar los comentarios
encoded_data = tokenizer.batch_encode_plus(
    comentarios,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='np'
)

# Datos de entrada y atención
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

# Verificar las dimensiones
print(f"Dimensiones de input_ids: {input_ids.shape}")
print(f"Dimensiones de encoded_labels: {encoded_labels.shape}")

# Si las dimensiones son compatibles
if input_ids.shape[0] == len(encoded_labels):
    # Dividir datos en conjuntos de entrenamiento y prueba
    train_inputs, test_inputs, train_labels, test_labels = train_test_split(
        input_ids,
        encoded_labels,
        test_size=0.2,
        random_state=42
    )
    train_masks, test_masks, _, _ = train_test_split(
        attention_masks,
        encoded_labels,
        test_size=0.2,
        random_state=42
    )
else:
    print("Las dimensiones de input_ids y encoded_labels no coinciden.")
    
# Cargar el modelo preentrenado BERT para clasificación de secuencias
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(set(encoded_labels))
)

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Entrenar el modelo
history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    epochs=3,
    batch_size=32
)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate([test_inputs, test_masks], test_labels, verbose=2)
print(f"Test accuracy: {test_accuracy}")

# Predicciones en el conjunto de prueba
predictions = model.predict([test_inputs, test_masks])
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# Reporte de clasificación
target_names = label_encoder.classes_
print(classification_report(test_labels, predicted_labels, target_names=target_names))

separado

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Convertir la Serie de Pandas a una lista de strings
comentarios = df['comment'].tolist()
categorias = df['num_cat'].tolist()

# Codificación de Etiquetas
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(categorias)

# Tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Codificar los comentarios
encoded_data = tokenizer.batch_encode_plus(
    comentarios,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='np'
)

# Datos de entrada y atención
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

# Verificar las dimensiones
print(f"Dimensiones de input_ids: {input_ids.shape}")
print(f"Dimensiones de encoded_labels: {encoded_labels.shape}")



Dimensiones de input_ids: (1008, 128)
Dimensiones de encoded_labels: (1008,)


In [ ]:
# Si las dimensiones son compatibles
if input_ids.shape[0] == len(encoded_labels):
    # Dividir datos en conjuntos de entrenamiento y prueba
    train_inputs, test_inputs, train_labels, test_labels = train_test_split(
        input_ids,
        encoded_labels,
        test_size=0.2,
        random_state=42
    )
    train_masks, test_masks, _, _ = train_test_split(
        attention_masks,
        encoded_labels,
        test_size=0.2,
        random_state=42
    )
else:
    print("Las dimensiones de input_ids y encoded_labels no coinciden.")

In [ ]:
# Cargar el modelo preentrenado BERT para clasificación de secuencias
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(set(encoded_labels))
)

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Entrenar el modelo
history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    epochs=3,
    batch_size=32
)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate([test_inputs, test_masks], test_labels, verbose=2)
print(f"Test accuracy: {test_accuracy}")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
26/26 [==============================] - 600s 20s/step - loss: 1.0792 - accuracy: 0.3871
Epoch 2/3
26/26 [==============================] - 495s 19s/step - loss: 0.9139 - accuracy: 0.5782
Epoch 3/3
26/26 [==============================] - 518s 20s/step - loss: 0.7120 - accuracy: 0.6985
7/7 - 31s - loss: 0.8034 - accuracy: 0.6089 - 31s/epoch - 4s/step
Test accuracy: 0.6089109182357788


In [ ]:
def get_class_name_from_index(idx):
    # Aquí debes implementar la lógica para obtener el nombre de clase correspondiente al índice
    # Por ejemplo, si tienes una lista de nombres de clases:
    class_names = ["ideologia politica", "machismo", "racismo"]  # Reemplaza esto con tus nombres reales de clases
    return class_names[idx]

# Crear la lista de nombres de clases
class_names = [get_class_name_from_index(idx) for idx in label_encoder.classes_]




In [ ]:
# Predicciones en el conjunto de prueba
predictions = model.predict([test_inputs, test_masks])
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# Reporte de clasificación
target_names = list(label_encoder.classes_)
class_names = [get_class_name_from_index(idx) for idx in label_encoder.classes_]
print(classification_report(test_labels, predicted_labels, target_names=class_names))

7/7 [==============================] - 27s 4s/step
                    precision    recall  f1-score   support

ideologia politica       0.51      0.62      0.56        74
          machismo       0.62      0.67      0.64        57
           racismo       0.78      0.55      0.64        71

          accuracy                           0.61       202
         macro avg       0.64      0.61      0.62       202
      weighted avg       0.64      0.61      0.61       202



In [ ]:
from transformers import pipeline

# Crear un pipeline de clasificación
classifier = pipeline("sentiment-analysis")

# Guardar el pipeline
bert_pipeline = "Users/usuario/Downloads/irunela/beto"
classifier.save_pretrained(bert_pipeline)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
from pathlib import Path  # Para manipular rutas
# Definir la ruta donde quieres guardar el modelo
ruta_guardado = Path("Users/usuario/Downloads/irunela/beto")

# Crear el directorio si no existe
ruta_guardado.mkdir(parents=True, exist_ok=True)

# Guardar el modelo y el tokenizador en esa ruta
model.save_pretrained(ruta_guardado)
tokenizer.save_pretrained(ruta_guardado)


('Users/usuario/Downloads/irunela/beto/tokenizer_config.json',
 'Users/usuario/Downloads/irunela/beto/special_tokens_map.json',
 'Users/usuario/Downloads/irunela/beto/vocab.txt',
 'Users/usuario/Downloads/irunela/beto/added_tokens.json')

In [ ]:
import joblib

# Crear el pipeline
classifier = pipeline("sentiment-analysis")

# Guardar el pipeline como un archivo usando joblib
ruta_guardado = "Users/usuario/Downloads/irunela/beto/pipeline.pkl"
joblib.dump(classifier, ruta_guardado)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


['Users/usuario/Downloads/irunela/beto/pipeline.pkl']

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Especificar el modelo al crear el pipeline
model_name = "bert-base-multilingual-cased"  # Reemplaza esto con el nombre de tu modelo
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Crear un pipeline con el modelo específico
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Guardar el pipeline
ruta_guardado = "Users/usuario/Downloads/irunela/beto/pipeline"
classifier.save_pretrained(ruta_guardado)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# #Definir el pipeline
pipeline_bert = Pipeline([
    ('tokenizer', BertTokenizer.from_pretrained("bert-base-multilingual-cased")),
    ('bert', SVC(probability=True)),
])
#utilizamos solamente 300 estimadores para agilizar el proceso durante la clase